In [1]:
import pyspark
from pyspark.sql.functions import *

In [2]:
# Import the PySpark module
from pyspark.sql import SparkSession

# config = pyspark.SparkConf().set('spark.rpc.message.maxSize', '256')

# # Create SparkSession object
# spark = SparkSession.builder \
#                     .master('local[*]') \
#                     .appName('test') \
#                     .getOrCreate(conf=config)

spark = SparkSession.builder \
        .appName('app_name') \
        .master('local[*]') \
        .config('spark.sql.execution.arrow.pyspark.enabled', True) \
        .config('spark.sql.session.timeZone', 'UTC') \
        .config('spark.driver.memory','32G') \
        .config('spark.ui.showConsoleProgress', True) \
        .config('spark.sql.repl.eagerEval.enabled', True) \
        .config('spark.rpc.message.maxSize', '256') \
        .getOrCreate()

In [3]:
print(spark.version)

3.1.1


In [4]:
movies_df = spark.read.load("./ml-latest-small/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("./ml-latest-small/ratings.csv", format='csv', header = True)
links_df = spark.read.load("./ml-latest-small/links.csv", format='csv', header = True)
tags_df = spark.read.load("./ml-latest-small/tags.csv", format='csv', header = True)

In [5]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [6]:
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [7]:
links_df.show(5)

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows



In [8]:
tags_df.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows



In [9]:
ratings_df.groupby('userID').count().select(min("count")).show()

+----------+
|min(count)|
+----------+
|        20|
+----------+



In [10]:
ratings_df.groupby('movieId').count().select(min("count")).show()

+----------+
|min(count)|
+----------+
|         1|
+----------+



In [11]:
ratings_df.groupby('movieId').count().filter(col('count') == 1).count()

3446

In [12]:
ratings_df.select('movieId').distinct().count()

9724

## Part 1: Spark SQL and OLAP

In [13]:
print(movies_df.columns)
print(ratings_df.columns)
print(links_df.columns)
print(tags_df.columns)

['movieId', 'title', 'genres']
['userId', 'movieId', 'rating', 'timestamp']
['movieId', 'imdbId', 'tmdbId']
['userId', 'movieId', 'tag', 'timestamp']


### Q1: The number of Users

In [14]:
ratings_df.select('userId').distinct().count()

610

### Q2: The number of Movies

In [15]:
movies_df.select('movieId').distinct().count()

9742

### Q3: How many movies are rated by users? List movies not rated before

In [16]:
movies_df.select('movieId').subtract(ratings_df.select('movieId')).count()

18

### Q4: List Movie Genres

In [17]:
movies_df.select('genres').distinct().show()

+--------------------+
|              genres|
+--------------------+
|Comedy|Horror|Thr...|
|Adventure|Sci-Fi|...|
|Action|Adventure|...|
| Action|Drama|Horror|
|Action|Animation|...|
|Animation|Childre...|
|Action|Adventure|...|
|    Adventure|Sci-Fi|
|Documentary|Music...|
|Adventure|Childre...|
| Adventure|Animation|
| Musical|Romance|War|
|Action|Adventure|...|
|Adventure|Childre...|
|Comedy|Crime|Horr...|
|Crime|Drama|Fanta...|
|Comedy|Mystery|Th...|
|   Adventure|Fantasy|
|Horror|Romance|Sc...|
|Drama|Film-Noir|R...|
+--------------------+
only showing top 20 rows



### Q5: Movie for Each Category

In [18]:
movies_df.select(split(col("genres"),"\|").alias("genres")).withColumn('genres', explode('genres')).groupby('genres').count().orderBy(col("count").desc()).show()

+------------------+-----+
|            genres|count|
+------------------+-----+
|             Drama| 4361|
|            Comedy| 3756|
|          Thriller| 1894|
|            Action| 1828|
|           Romance| 1596|
|         Adventure| 1263|
|             Crime| 1199|
|            Sci-Fi|  980|
|            Horror|  978|
|           Fantasy|  779|
|          Children|  664|
|         Animation|  611|
|           Mystery|  573|
|       Documentary|  440|
|               War|  382|
|           Musical|  334|
|           Western|  167|
|              IMAX|  158|
|         Film-Noir|   87|
|(no genres listed)|   34|
+------------------+-----+



## Part2: Spark ALS based approach for training model

In [19]:
print(movies_df.columns)
print(ratings_df.columns)
print(links_df.columns)
print(tags_df.columns)

['movieId', 'title', 'genres']
['userId', 'movieId', 'rating', 'timestamp']
['movieId', 'imdbId', 'tmdbId']
['userId', 'movieId', 'tag', 'timestamp']


In [20]:
ratings_df = spark.read.load("./ml-latest-small/ratings1.csv", format='csv', header = True)

In [21]:
ratings_df = ratings_df.filter(col('movieId') < 100)

In [22]:
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|     4|964982703|
|     2|      2|     4|964981247|
|     3|      3|     3|964982224|
|     4|      4|     5|964983815|
|     5|      1|     1|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [23]:
# Count the total number of ratings in the dataset
numerator = ratings_df.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings_df.select("userId").distinct().count()
num_movies = ratings_df.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  -200.00% empty.


In [24]:
# Filter to show only userIds less than 100
ratings_df.filter(col("userId") < 100).show()

# Group data by userId, count ratings
ratings_df.groupBy("userId").count().show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|     4|964982703|
|     2|      2|     4|964981247|
|     3|      3|     3|964982224|
|     4|      4|     5|964983815|
|     5|      1|     1|964982931|
|     6|      2|     3|964982400|
|     7|      3|     5|964980868|
|     1|      4|     4|964982176|
|     2|      1|     5|964984041|
|     3|      2|     2|964984100|
|     4|      3|     5|964983650|
|     5|      4|     5|964981208|
|     6|      1|     2|964980985|
|     7|      2|     5|964981179|
|     1|      3|     1|964980908|
|     2|      4|     4|964982703|
|     3|      1|     4|964981247|
|     4|      2|     3|964982224|
|     5|      3|     5|964983815|
|     6|      4|     1|964982931|
+------+-------+------+---------+
only showing top 20 rows

+------+-----+
|userId|count|
+------+-----+
|     7|   12|
|     3|   12|
|     5|   12|
|     6|   12|
|     1|   12|
|     4|   12|
|     2|   12|
+------+

In [25]:
# Min num ratings for movies
print("Movie with the fewest ratings: ")
ratings_df.groupBy("movieId").count().select(min("count")).show()

# Avg num ratings per movie
print("Avg num ratings per movie: ")
ratings_df.groupBy("movieId").count().select(avg("count")).show()

# Min num ratings for user
print("User with the fewest ratings: ")
ratings_df.groupBy("userId").count().select(min("count")).show()

# Avg num ratings per users
print("Avg num ratings per user: ")
ratings_df.groupBy("userId").count().select(avg("count")).show()

Movie with the fewest ratings: 
+----------+
|min(count)|
+----------+
|        21|
+----------+

Avg num ratings per movie: 
+----------+
|avg(count)|
+----------+
|      21.0|
+----------+

User with the fewest ratings: 
+----------+
|min(count)|
+----------+
|        12|
+----------+

Avg num ratings per user: 
+----------+
|avg(count)|
+----------+
|      12.0|
+----------+



In [26]:
# Use .printSchema() to see the datatypes of the ratings dataset
ratings_df.printSchema()

# Tell Spark to convert the columns to the proper data types
ratings_df = ratings_df.select(ratings_df.userId.cast("integer"), ratings_df.movieId.cast("integer"), ratings_df.rating.cast("double"))

# Call .printSchema() again to confirm the columns are now in the correct format
ratings_df.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [27]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create test and train set
(train, test) = ratings_df.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False)

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [28]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50]) \
            .addGrid(als.maxIter, [5, 50]) \
            .addGrid(als.regParam, [.01, .1]) \
            .build()
           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  8


In [29]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=2)

# Confirm cv was built
print(cv)

CrossValidator_78bd27626e32


In [30]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

Py4JJavaError: An error occurred while calling o695.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 225.0 failed 1 times, most recent failure: Lost task 0.0 in stage 225.0 (TID 4628) (host.docker.internal executor driver): java.lang.StackOverflowError
	at java.lang.reflect.Constructor.newInstance(Unknown Source)
	at java.io.ObjectStreamClass.newInstance(Unknown Source)
	at java.io.ObjectInputStream.readOrdinaryObject(Unknown Source)
	at java.io.ObjectInputStream.readObject0(Unknown Source)
	at java.io.ObjectInputStream.defaultReadFields(Unknown Source)
	at java.io.ObjectInputStream.readSerialData(Unknown Source)
	at java.io.ObjectInputStream.readOrdinaryObject(Unknown Source)
	at java.io.ObjectInputStream.readObject0(Unknown Source)
	at java.io.ObjectInputStream.readObject(Unknown Source)
	at java.io.ObjectInputStream.readObject(Unknown Source)
	at scala.collection.immutable.List$SerializationProxy.readObject(List.scala:488)
	at sun

In [ ]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# Print "Rank"
print("  Rank:", best_model.getRank())

# Print "MaxIter"
print("  MaxIter:", best_model.getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model.getRegParam())